# स्थिर मेमरीसह AI एजंट तयार करणे Mem0, Semantic Kernel, आणि Azure AI Search वापरून

या नोटबुकमध्ये एक बुद्धिमान प्रवास बुकिंग एजंट तयार करण्याचे प्रदर्शन केले आहे जो वापरकर्त्याच्या संवादांमध्ये प्राधान्ये लक्षात ठेवतो. Mem0, Semantic Kernel, आणि Azure AI Search एकत्र करून, आम्ही एक असा एजंट तयार करतो जो ऐतिहासिक संवादांवर आधारित वैयक्तिकृत प्रवास शिफारसी प्रदान करतो.

## तुम्ही काय शिकाल:
1. **Mem0 एकत्रीकरण**: AI एजंटसाठी Mem0 मेमरी स्तर म्हणून कसे वापरायचे
2. **Azure AI Search as Vector Store**: मेमरी साठवणे आणि सेमॅंटिक शोध वापरून पुनर्प्राप्त करणे
3. **स्थिर वापरकर्ता प्राधान्ये**: वेगवेगळ्या चॅट सत्रांमध्ये वापरकर्त्याची प्राधान्ये लक्षात ठेवणे
4. **Semantic Kernel प्लगइन्स**: मेमरी आणि शोध क्षमता दोन्ही वापरणारे प्लगइन्स तयार करणे

## पूर्वअट:
- Azure OpenAI डिप्लॉयमेंट कॉन्फिगर केलेले
- Azure AI Search सेवा तयार केलेली
- मूलभूत Semantic Kernel संकल्पनांची समज


## मेमरी आर्किटेक्चर समजून घेणे

### Mem0 म्हणजे काय?

**Mem0** ही एक बुद्धिमान मेमरी लेयर आहे जी खालील सुविधा प्रदान करते:
- **दीर्घकालीन मेमरी**: वापरकर्त्याच्या पसंती, पूर्वीच्या संवाद आणि शिकलेली माहिती साठवणे
- **सामान्य शोध**: संदर्भानुसार संबंधित मेमरी शोधणे
- **वापरकर्त्यासाठी विशिष्ट साठवण**: वेगवेगळ्या वापरकर्त्यांसाठी स्वतंत्र मेमरी जागा राखणे
- **स्वयंचलित प्रासंगिकता**: सध्याच्या संदर्भासाठी सर्वात संबंधित मेमरी समोर आणणे

### घटक कसे एकत्र काम करतात:
```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│  Semantic       │────▶│      Mem0        │────▶│  Azure AI       │
│  Kernel Agent   │     │  Memory Layer    │     │  Search         │
└─────────────────┘     └──────────────────┘     └─────────────────┘
         │                       │                         │
         │                       │                         │
    Processes              Stores/Retrieves          Vector Store
    User Input             User Preferences         for Memories &
                          & Context                  Travel Data
```


In [ ]:
! pip install mem0ai

## आवश्यक पॅकेजेस आयात करा


In [ ]:
import json
import os
from typing import Annotated, List, Dict, Any
from datetime import datetime
import uuid

from IPython.display import display, HTML, Markdown
from dotenv import load_dotenv

# Azure AI Search
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchField,
    VectorSearchAlgorithmMetric
)

# Mem0
from mem0 import Memory

# Semantic Kernel
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatHistory
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

## पर्यावरण कॉन्फिगरेशन


In [ ]:
# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")  # Use a recent API version


# Azure AI Search Configuration
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")

# Index names
travel_index_name = "travel-hotels"
memory_index_name = "mem0-memories"



## प्रवास डेटा साठी Azure AI Search प्रारंभ करा

सर्वप्रथम, आपण Azure AI Search सेट अप करू, ज्यामध्ये आमचा एजंट शोध घेऊ शकेल अशा नमुना हॉटेल आणि गंतव्य डेटा समाविष्ट असेल.


In [ ]:
# Initialize search clients
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Create travel data index if it doesn't exist
travel_fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="name", type=SearchFieldDataType.String),
    SearchableField(name="description", type=SearchFieldDataType.String),
    SearchableField(name="location", type=SearchFieldDataType.String),
    SearchableField(name="amenities", type=SearchFieldDataType.String),
    SimpleField(name="price_per_night", type=SearchFieldDataType.Double),
    SimpleField(name="rating", type=SearchFieldDataType.Double),
    SearchableField(name="tags", type=SearchFieldDataType.String, collection=True)
]

travel_index = SearchIndex(name=travel_index_name, fields=travel_fields)

try:
    index_client.get_index(travel_index_name)
    print(f"✅ Index '{travel_index_name}' already exists")
except:
    index_client.create_index(travel_index)
    print(f"✅ Created index '{travel_index_name}'")

# Initialize search client for travel data
travel_search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=travel_index_name,
    credential=AzureKeyCredential(search_api_key)
)

In [ ]:
# Add sample travel data
sample_hotels = [
    {
        "id": "1",
        "name": "Le Meurice Paris",
        "description": "Luxury palace hotel with Michelin-starred dining and views of the Tuileries Garden",
        "location": "Paris, France",
        "amenities": "Spa, Michelin Restaurant, Concierge, Room Service, Fitness Center",
        "price_per_night": 850,
        "rating": 4.8,
        "tags": ["luxury", "romantic", "historic", "fine-dining", "spa"]
    },
    {
        "id": "2",
        "name": "Four Seasons Maui",
        "description": "Beachfront resort with world-class spa and family-friendly activities",
        "location": "Maui, Hawaii",
        "amenities": "Beach Access, Kids Club, Multiple Pools, Spa, Golf Course",
        "price_per_night": 695,
        "rating": 4.7,
        "tags": ["beach", "family-friendly", "resort", "spa", "golf"]
    },
    {
        "id": "3",
        "name": "Aman Tokyo",
        "description": "Minimalist luxury hotel with panoramic city views and traditional onsen",
        "location": "Tokyo, Japan",
        "amenities": "Onsen, City Views, Fine Dining, Spa, Business Center",
        "price_per_night": 780,
        "rating": 4.9,
        "tags": ["luxury", "business", "spa", "city", "minimalist"]
    },
    {
        "id": "4",
        "name": "Hotel Sacher Vienna",
        "description": "Historic hotel home of the original Sachertorte with elegant rooms",
        "location": "Vienna, Austria",
        "amenities": "Historic Cafe, Concierge, Accessible Rooms, Pet-Friendly",
        "price_per_night": 420,
        "rating": 4.6,
        "tags": ["historic", "accessible", "pet-friendly", "cultural", "cafe"]
    },
    {
        "id": "5",
        "name": "Fairmont Whistler",
        "description": "Ski-in/ski-out resort with family suites and mountain views",
        "location": "Whistler, Canada",
        "amenities": "Ski Access, Family Suites, Heated Pool, Kids Programs",
        "price_per_night": 380,
        "rating": 4.5,
        "tags": ["ski", "family-friendly", "mountain", "resort", "accessible"]
    }
]

# Upload hotels to search index
travel_search_client.upload_documents(documents=sample_hotels)
print(f"✅ Uploaded {len(sample_hotels)} hotels to search index")

## Azure AI Search सह Mem0 प्रारंभ करा

आता आपण Mem0 कॉन्फिगर करू जेणेकरून ते स्थायी मेमरीसाठी Azure AI Search ला त्याच्या व्हेक्टर स्टोअर म्हणून वापरू शकेल.


In [ ]:
mem0_config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": azure_openai_deployment,
            "temperature": 0.2,
            "max_tokens": 1500,
            "azure_kwargs": {
                "azure_deployment": azure_openai_deployment,
                "api_version": api_version,
                "azure_endpoint": azure_openai_endpoint,
                "api_key": azure_openai_api_key,
            }
        }
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": search_service_endpoint.split("//")[1].split(".")[0],
            "api_key": search_api_key,
            "collection_name": "mem0",
            "embedding_model_dims": 1536
        }
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": "text-embedding-ada-002",  # Your embedding deployment name
            "azure_kwargs": {
                "azure_deployment": "text-embedding-ada-002",  # Update if different
                "api_version": api_version,
                "azure_endpoint": azure_openai_endpoint,
                "api_key": azure_openai_api_key,
            }
        }
    }
}

# Initialize Mem0
memory = Memory.from_config(mem0_config)
# Test the memory system
print("🧪 Testing Mem0 setup...")
test_messages = [
    {"role": "user", "content": "I prefer luxury hotels with spa services."},
    {"role": "assistant", "content": "I'll remember you prefer luxury hotels with spa services for future recommendations."}
]
memory.add(test_messages, user_id="test_user",
           metadata={"category": "preferences"})
test_memories = memory.get_all(user_id="test_user")
print(f"✅ Mem0 test successful! Found {len(test_memories)} memories")

## प्रवास बुकिंग प्लगइन तयार करा

हा प्लगइन हॉटेल शोधण्यासाठी आणि Mem0 च्या माध्यमातून वापरकर्त्याच्या पसंती व्यवस्थापित करण्यासाठी फंक्शन्स प्रदान करतो.


In [ ]:
class TravelBookingPlugin:
    """Plugin for searching hotels and managing user travel preferences"""

    def __init__(self, search_client: SearchClient, memory: Memory):
        self.search_client = search_client
        self.memory = memory

    @kernel_function(
        description="Search for hotels based on criteria like location, amenities, or tags"
    )
    def search_hotels(
        self,
        query: Annotated[str, "Search query for hotels (location, amenities, etc.)"],
        max_results: Annotated[int, "Maximum number of results to return"] = 3
    ) -> Annotated[str, "List of hotels matching the search criteria"]:
        """Search for hotels in the travel database"""
        results = self.search_client.search(
            search_text=query,
            top=max_results,
            include_total_count=True
        )

        hotels = []
        for result in results:
            hotels.append({
                "name": result["name"],
                "location": result["location"],
                "description": result["description"],
                "price_per_night": result["price_per_night"],
                "rating": result["rating"],
                "amenities": result["amenities"],
                "tags": result["tags"]
            })

        return json.dumps(hotels, indent=2)

    @kernel_function(
        description="Store user travel preferences and important information in memory"
    )
    def store_user_preference(
        self,
        user_id: Annotated[str, "User identifier"],
        preference: Annotated[str,
                              "User preference or information to remember"]
    ) -> Annotated[str, "Confirmation of stored preference"]:
        """Store user preferences in Mem0 memory"""
        print(f"DEBUG: Storing preference for {user_id}: {preference}")

        try:
            # Simply add the preference to memory
            self.memory.add(preference, user_id=user_id)
            return f"✅ Stored: {preference}"
        except Exception as e:
            return f"❌ Error storing preference: {str(e)}"
        
    @kernel_function(
        description="Get all stored preferences for a user"
    )
    def get_user_preferences(
        self,
        user_id: Annotated[str, "User identifier"]
    ) -> Annotated[str, "All user preferences and memories"]:
        """Get all memories for a specific user"""
        print(f"DEBUG: Getting all preferences for {user_id}")

        try:
            # Get all memories for the user
            results = self.memory.get_all(user_id=user_id)

            # Handle the dict response with 'results' key
            if isinstance(results, dict) and 'results' in results:
                results = results.get('results', [])

            if not results:
                return f"No preferences found for user {user_id}"

            # Format results
            memories = []
            for result in results:
                if isinstance(result, dict):
                    memory_text = result.get('memory', str(result))
                    memories.append(memory_text)
                else:
                    memories.append(str(result))

            return f"User preferences for {user_id}:\n- " + "\n- ".join(memories)

        except Exception as e:
            print(f"ERROR getting preferences: {str(e)}")
            return f"No preferences found for user {user_id}"


    @kernel_function(
        description="Search user's memories for relevant information"
    )
    def search_memories(
        self,
        user_id: Annotated[str, "User identifier"],
        query: Annotated[str,
                         "What to search for (e.g., 'family vacation', 'dietary restrictions')"]
    ) -> Annotated[str, "Relevant memories"]:
        """Search user memories using Mem0"""
        print(f"DEBUG: Searching memories for {user_id} with query: '{query}'")

        try:
            # Let Mem0 handle the search and ranking
            results = self.memory.search(query, user_id=user_id)

            # Handle the dict response with 'results' key
            if isinstance(results, dict) and 'results' in results:
                results = results.get('results', [])

            if not results:
                return f"No memories found for query: {query}"

            # Format results
            memories = []
            for result in results:
                if isinstance(result, dict):
                    memory_text = result.get('memory', str(result))
                    # Include relevance score if available
                    score = result.get('score', None)
                    if score:
                        memories.append(
                            f"{memory_text} (relevance: {score:.2f})")
                    else:
                        memories.append(memory_text)
                else:
                    memories.append(str(result))

            return "Relevant memories:\n- " + "\n- ".join(memories)

        except Exception as e:
            print(f"ERROR: {str(e)}")
            return "No memories found."
        
        
    

## सेमॅंटिक कर्नल एजंट प्रारंभ करा

ट्रॅव्हल बुकिंग प्लगइनसह प्रवेशासह आमचा ट्रॅव्हल बुकिंग एजंट तयार करा.


In [ ]:
# Create the kernel
kernel = Kernel()

# Add Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=azure_openai_deployment,
    endpoint=azure_openai_endpoint,
    api_key=azure_openai_api_key,
)
kernel.add_service(chat_service)

# Create and add the travel booking plugin
travel_plugin = TravelBookingPlugin(travel_search_client, memory)
kernel.add_plugin(
    plugin_name="TravelBooking",
    plugin=travel_plugin
)

# Create the travel agent
# Create the travel agent
travel_agent = ChatCompletionAgent(
    service=chat_service,
    name="TravelBookingAssistant",
    instructions="""
    You are a personalized travel booking assistant with memory.
    
    WORKFLOW:
    1. When a user asks for help, search their memories using search_memories() with a relevant query
    2. Use the memories to personalize your response
    3. Store any new preferences they mention using store_user_preference()
    4. When the users is booking a new trip, first retrieve the users general travel preferences of the user by creating queries for hotels, dietary restrictions, location, amenities and budget. THEN use search_hotels() to find suitable options.
    5. Do not recommend hotels that are over budget. 
    
    IMPORTANT: For ALL memory operations (search_memories and store_user_preference), 
    you MUST use user_id='sarah_johnson_123' exactly as written.

    Example queries:
    - User asks about booking a trip → search_memories(query="preferences")
    - User asks about booking a trip → search_memories(query="dietary restrictions")
    - User asks about booking a trip → search_memories(query="location")
    - User asks about booking a trip → search_memories(query="amenities")
    - User asks about booking a trip → search_memories(query="budget")

    Always acknowledge what you found in their memories when responding.""",
    plugins=[travel_plugin]
)

## स्वच्छ प्रदर्शनासाठी सहाय्यक फंक्शन्स


In [ ]:
def display_message(role: str, content: str, color: str = "#2E8B57", emoji: str = ""):
    """Display a message with nice formatting"""
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 15px 20px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.05); 
        border-radius: 8px;
    '>
        <strong style='color: {color}; font-size: 16px;'>{emoji} {role}:</strong><br>
        <div style='margin-top: 10px; white-space: pre-wrap; font-size: 14px; line-height: 1.6;'>{content}</div>
    </div>
    """
    display(HTML(html))

def display_memory_operation(operation: str, details: str, color: str = "#9370DB"):
    """Display memory operations for educational purposes"""
    html = f"""
    <div style='
        margin: 5px 20px;
        padding: 10px 15px;
        background: rgba(147, 112, 219, 0.1);
        border: 1px solid {color};
        border-radius: 6px;
        font-family: monospace;
        font-size: 13px;
    '>
        <strong style='color: {color};'>🧠 Memory {operation}:</strong>
        <div style='margin-top: 5px; color: #555;'>{details}</div>
    </div>
    """
    display(HTML(html))

def display_function_call(function_name: str, args: dict, result: str = None):
    """Display function calls for transparency"""
    html = f"""
    <details style='margin: 5px 20px; padding: 10px; background: rgba(0, 123, 255, 0.05); border: 1px solid #007BFF; border-radius: 6px;'>
        <summary style='cursor: pointer; font-weight: bold; color: #007BFF;'>⚙️ Function Call: {function_name}</summary>
        <div style='margin-top: 10px; font-family: monospace; font-size: 12px;'>
            <div><strong>Arguments:</strong> {json.dumps(args, indent=2)}</div>
    """
    if result:
        html += f"<div style='margin-top: 10px;'><strong>Result:</strong><pre style='background: #f8f8f8; padding: 8px; border-radius: 4px; overflow-x: auto;'>{result}</pre></div>"
    html += "</div></details>"
    display(HTML(html))

## मेमरीसह प्रवास बुकिंगचे प्रदर्शन

चला, वापरकर्त्याच्या पसंती लक्षात ठेवून आणि वापरून एजंट कसा काम करतो हे दाखवणाऱ्या वास्तववादी प्रवास बुकिंग परिस्थितींचा आढावा घेऊया.


### प्रथमच वापरणारा - वर्धापनदिन प्रवास योजना


In [ ]:
# User ID for our demonstration
sarah_user_id = "sarah_johnson_123"

print("🎯 SCENARIO 1: Sarah's First Booking - Anniversary Trip\n")

# Create a new chat history for Sarah
sarah_chat = ChatHistoryAgentThread()

# First conversation
user_message1 = """Hi! I'm Sarah and I'm planning a special trip for my 10th wedding anniversary. 
We love romantic destinations, fine dining, and spa experiences. My husband has mobility issues, 
so we need accessible accommodations. Our budget is around $700-800 per night."""

display_message("Sarah", user_message1, "#4fc3f7", "👤")

# Extract and display function calls for educational purposes
# Process with agent
response_content = ""
function_calls_made = []

async for response in travel_agent.invoke(
    messages=user_message1,
    thread=sarah_chat
):
    if response.content:
        response_content = str(response.content)

    # Parse function calls from the thread
    if hasattr(response, 'thread'):
        sarah_thread = response.thread

        # Check for function calls in the latest messages
        async for msg in sarah_thread.get_messages():
            if hasattr(msg, 'items') and msg.items:
                for item in msg.items:
                    if hasattr(item, 'function_invoke_attempt') and item.function_invoke_attempt:
                        func_call = item.function_invoke_attempt
                        function_info = {
                            'name': func_call.function_name,
                            'arguments': func_call.arguments,
                            'result': item.function_result.value if hasattr(item, 'function_result') else None
                        }
                        if function_info not in function_calls_made:
                            function_calls_made.append(function_info)

                            # Display the actual function calls
                            if 'get_user_preferences' in func_call.function_name:
                                display_memory_operation(
                                    "Retrieval", f"Checking existing preferences for user: {func_call.arguments.get('user_id', sarah_user_id)}")
                            elif 'store_user_preference' in func_call.function_name:
                                display_memory_operation(
                                    "Storage", f"Storing: {func_call.arguments.get('preference', '')}")
                            elif 'search_hotels' in func_call.function_name:
                                display_function_call(
                                    func_call.function_name,
                                    func_call.arguments,
                                    item.function_result.value if hasattr(
                                        item, 'function_result') else None
                                )

display_message("Travel Assistant", response_content, "#81c784", "🤖")

In [ ]:
user_message2 = """The Hotel Sacher sounds perfect! We're both vegetarian and I have a severe nut allergy. 
Can you tell me more about their dining options?"""

display_message("Sarah", user_message2, "#4fc3f7", "👤")

response2_content = ""
async for response in travel_agent.invoke(
    messages=user_message2,
    thread=sarah_thread
):
    if response.content:
        response2_content = str(response.content)

    if hasattr(response, 'thread'):
        sarah_thread = response.thread

        # Parse new function calls
        async for msg in sarah_thread.get_messages():
            if hasattr(msg, 'items') and msg.items:
                for item in msg.items:
                    if hasattr(item, 'function_invoke_attempt') and item.function_invoke_attempt:
                        func_call = item.function_invoke_attempt
                        if 'store_user_preference' in func_call.function_name and func_call.arguments.get('preference'):
                            pref = func_call.arguments.get('preference', '')
                            if 'vegetarian' in pref.lower() or 'nut allergy' in pref.lower():
                                display_memory_operation(
                                    "Storage", f"Storing: {pref}")

display_message("Travel Assistant", response2_content, "#81c784", "🤖")

In [ ]:

# After running all scenarios, verify memories are stored
from azure.search.documents import SearchClient
print("\n\n🔍 VERIFYING MEM0 STORAGE\n")

# Check Azure AI Search directly
mem0_search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name="mem0",
    credential=AzureKeyCredential(search_api_key)
)

try:
    # Count documents in the index
    results = mem0_search_client.search(
        search_text="*", include_total_count=True)
    total_docs = results.get_count()
    print(f"📊 Total documents in Mem0 index: {total_docs}")

    # Show first few documents
    print("\nSample documents:")
    for i, doc in enumerate(results):
        if i < 3:  # Show first 3
            print(f"\nDocument {i+1}:")
            print(f"  ID: {doc.get('id', 'N/A')}")
            print(f"  User ID: {doc.get('user_id', 'N/A')}")
            print(f"  Memory: {doc.get('payload', 'N/A')}")
except Exception as e:
    print(f"❌ Error checking Mem0 index: {str(e)}")
    print("The index might not be created yet or might be empty.")

In [ ]:
# Enhanced verification to debug Mem0 responses
print("🔍 ENHANCED MEM0 VERIFICATION\n")

# Create a unique test user
test_user = f"debug_user_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
test_memory = "I am vegetarian with a peanut allergy and love beach destinations"

print(f"1. Adding memory for {test_user}...")
add_result = memory.add(test_memory, user_id=test_user)
print(f"   Raw add result: {add_result}")

# Check if the result has a 'results' key
if isinstance(add_result, dict) and 'results' in add_result:
    actual_results = add_result.get('results', [])
    print(
        f"   Actual memories added: {len(actual_results) if isinstance(actual_results, list) else 0}")
    if isinstance(actual_results, list):
        for mem in actual_results:
            print(
                f"   - ID: {mem.get('id', 'N/A')}, Memory: {mem.get('memory', 'N/A')}")

print("\n2. Testing get_all()...")
all_mems = memory.get_all(user_id=test_user)
print(f"   Raw response: {all_mems}")
print(f"   Response type: {type(all_mems)}")

# Check if it's a dict with 'results' key
if isinstance(all_mems, dict):
    print(f"   Dict keys: {list(all_mems.keys())}")
    if 'results' in all_mems:
        results_value = all_mems['results']
        print(f"   'results' value type: {type(results_value)}")
        print(f"   'results' value: {results_value}")

        # If results is actually a list, show the memories
        if isinstance(results_value, list):
            print(f"   Number of memories: {len(results_value)}")
            for i, mem in enumerate(results_value):
                print(f"   Memory {i}: {mem}")

print("\n3. Testing search()...")
search_results = memory.search("peanut allergy", user_id=test_user)
print(f"   Raw response: {search_results}")
print(f"   Response type: {type(search_results)}")

# Check if it's a dict with 'results' key
if isinstance(search_results, dict):
    print(f"   Dict keys: {list(search_results.keys())}")
    if 'results' in search_results:
        results_value = search_results['results']
        print(f"   'results' value type: {type(results_value)}")
        print(f"   'results' value: {results_value}")

print("\n4. Testing direct API access...")
# Try to access memories through Azure AI Search directly
try:
    mem0_search_client = SearchClient(
        endpoint=search_service_endpoint,
        index_name="mem0",
        credential=AzureKeyCredential(search_api_key)
    )

    # Wait a moment for indexing
    import time
    time.sleep(2)

    # Search for the test user's memories
    azure_results = mem0_search_client.search(
        search_text="*",
        filter=f"user_id eq '{test_user}'",
        include_total_count=True
    )

    print(f"   Documents found in Azure: {azure_results.get_count()}")
    for doc in azure_results:
        payload = doc.get('payload', {})
        if isinstance(payload, str):
            import json
            try:
                payload = json.loads(payload)
            except:
                pass
        print(f"   - Memory: {payload}")

except Exception as e:
    print(f"   Error: {e}")

print("\n5. Testing Mem0 version...")
# Check if we need to use a different method or property
if hasattr(memory, '__version__'):
    print(f"   Mem0 version: {memory.__version__}")
if hasattr(memory, 'version'):
    print(f"   Mem0 version: {memory.version}")

# Check available methods
print("\n6. Available memory methods:")
for attr in dir(memory):
    if not attr.startswith('_') and callable(getattr(memory, attr)):
        print(f"   - {attr}")

### परिस्थिती २: परत भेट - कौटुंबिक सुट्टी (काही आठवड्यांनंतर)


In [ ]:
print("\n\n🎯 SCENARIO 2: Sarah Returns Weeks Later for Family Vacation\n")
print("📅 Simulating time passing... Sarah starts a new conversation\n")

# Create a new thread to simulate a new conversation
sarah_thread_new = ChatHistoryAgentThread()

user_message3 = "Hi, my husband and I are planning another trip. We are looking for a good hotel!"

display_message("Sarah", user_message3, "#4fc3f7", "👤")

response3_content = ""
memories_retrieved = []

async for response in travel_agent.invoke(
    messages=user_message3,
    thread=sarah_thread_new
):
    if response.content:
        response3_content = str(response.content)

    if hasattr(response, 'thread'):
        sarah_thread_new = response.thread

        # Parse function calls
        async for msg in sarah_thread_new.get_messages():
            if hasattr(msg, 'items') and msg.items:
                for item in msg.items:
                    if hasattr(item, 'function_invoke_attempt') and item.function_invoke_attempt:
                        func_call = item.function_invoke_attempt

                        # Check for memory retrieval
                        if 'get_user_preferences' in func_call.function_name and hasattr(item, 'function_result'):
                            result = item.function_result.value
                            if result and "User preferences" in result:
                                display_memory_operation(
                                    "Retrieval", f"Found memories for {sarah_user_id}:\n{result}")

                        # Check for new preference storage
                        elif 'store_user_preference' in func_call.function_name:
                            display_memory_operation(
                                "Storage", f"Storing: {func_call.arguments.get('preference', '')}")

                        # Check for hotel search
                        elif 'search_hotels' in func_call.function_name:
                            display_function_call(
                                func_call.function_name,
                                func_call.arguments,
                                item.function_result.value if hasattr(
                                    item, 'function_result') else None
                            )

display_message("Travel Assistant", response3_content, "#81c784", "🤖")

In [ ]:
# Follow-up question
user_message4 = "Great suggestions! For the Maui option, what activities would you recommend for the kids?"

display_message("Sarah", user_message4, "#4fc3f7", "👤")

response4_content = ""
async for response in travel_agent.invoke(
    messages=user_message4,
    thread=sarah_thread_new
):
    if response.content:
        response4_content = str(response.content)
    sarah_thread_new = response.thread

display_message("Travel Assistant", response4_content, "#81c784", "🤖")

In [ ]:
print("\n🧪 TESTING MEMORY RETRIEVAL\n")

# First, ensure Sarah has some memories
test_preference = "I love romantic destinations with spa services"
result = travel_plugin.store_user_preference(sarah_user_id, test_preference)
print(f"Store result: {result}")

# Now test retrieval
preferences = travel_plugin.get_user_preferences(sarah_user_id)
print(f"\nRetrieved preferences:\n{preferences}")

# Also test the memory object directly
direct_memories = memory.get_all(user_id=sarah_user_id)
print(f"\nDirect memory.get_all() returned {len(direct_memories)} memories")
for i, mem in enumerate(direct_memories):
    print(f"Memory {i}: {mem}")

In [ ]:

# Check the Mem0 index structure in Azure AI Search
print("\n🔍 CHECKING MEM0 INDEX STRUCTURE\n")

try:
    # Get the mem0 index
    mem0_index = index_client.get_index("mem0")
    print("Mem0 index fields:")
    for field in mem0_index.fields:
        print(f"  - {field.name}: {field.type}")

    # Query the index directly
    mem0_search_client = SearchClient(
        endpoint=search_service_endpoint,
        index_name="mem0",
        credential=AzureKeyCredential(search_api_key)
    )

    # Get all documents for Sarah
    results = mem0_search_client.search(
        search_text="*",
        filter=f"user_id eq '{sarah_user_id}'",
        include_total_count=True
    )

    print(f"\nDocuments for {sarah_user_id}: {results.get_count()}")
    for doc in results:
        print(f"\nDocument ID: {doc.get('id')}")
        for key, value in doc.items():
            if key != 'id':
                print(
                    f"  {key}: {value[:100] if isinstance(value, str) and len(value) > 100 else value}")

except Exception as e:
    print(f"Error checking index: {str(e)}")

## सेमॅंटिक मेमरी शोधाचे प्रात्यक्षिक

Mem0 ची ताकद सेमॅंटिक शोधामध्ये आहे - केवळ कीवर्ड्सवर आधारित नसून, अर्थावर आधारित संबंधित आठवणी शोधण्यात.


In [ ]:
print("🔍 SEMANTIC MEMORY SEARCH DEMONSTRATION\n")

# Search Sarah's memories for dietary-related information
dietary_search = memory.search(
    "dietary food allergies restrictions", user_id=sarah_user_id)

# Handle the dict response with 'results' key
if isinstance(dietary_search, dict) and 'results' in dietary_search:
    dietary_results = dietary_search.get('results', [])
else:
    dietary_results = dietary_search if isinstance(
        dietary_search, list) else []

print("Search Query: 'dietary food allergies restrictions'")
print(f"Results for Sarah:")
print("=" * 50)
if dietary_results:
    for mem in dietary_results:
        if isinstance(mem, dict):
            print(f"- {mem.get('memory', 'Unknown')}")
            print(f"  Relevance Score: {mem.get('score', 'N/A')}")
        else:
            print(f"- {mem}")
else:
    print("- No memories found")



## मुख्य मुद्दे

### 1. सतत वापरकर्ता मेमरी
- **सत्रांमधील सातत्य**: वापरकर्त्याच्या पसंती वेगवेगळ्या संभाषणांमध्ये जतन केल्या जातात
- **वापरकर्ता वेगळेपणा**: प्रत्येक वापरकर्त्याला स्वतःची स्वतंत्र मेमरी जागा असते
- **स्वयंचलित संदर्भ**: एजंट संबंधित मेमरी आपोआप पुनर्प्राप्त करतो

### 2. Mem0 चे फायदे
- **अर्थपूर्ण समज**: अचूक जुळणीवर आधारित नसून अर्थावर आधारित मेमरी पुनर्प्राप्त करते
- **स्केलेबिलिटी**: एंटरप्राइझ-ग्रेड स्टोरेजसाठी Azure AI Search वापरते
- **गोपनीयता**: वापरकर्त्याच्या मेमरी वेगळ्या आणि सुरक्षित असतात

### 3. सुधारित वापरकर्ता अनुभव
- **पुनरावृत्ती नाही**: वापरकर्त्यांना त्यांच्या पसंती पुन्हा सांगण्याची गरज नाही
- **वैयक्तिकरण**: शिफारसी वेळोवेळी सुधारतात
- **संदर्भ जागरूकता**: एजंट वापरकर्त्याच्या इतिहासाची समज ठेवतो


## सारांश

अभिनंदन! तुम्ही यशस्वीपणे एक AI प्रवास एजंट तयार केला आहे ज्यामध्ये स्थिर मेमरी क्षमता आहे, वापरून:

- **Mem0**: बुद्धिमान, स्थिर मेमरी व्यवस्थापनासाठी
- **Azure AI Search**: मेमरी आणि प्रवास डेटा साठी स्केलेबल व्हेक्टर स्टोअर म्हणून
- **Semantic Kernel**: एजंट आणि प्लगिन्सचे समन्वय साधण्यासाठी

## तुम्ही काय शिकले:
1. Mem0 आणि Azure AI Search यांचे स्थिर मेमरीसाठी एकत्रीकरण कसे करायचे
2. मेमरीचा उपयोग करणारे Semantic Kernel प्लगिन्स तयार करणे
3. असे एजंट तयार करणे जे सत्रांमध्ये वापरकर्त्याच्या पसंती लक्षात ठेवतात
4. संबंधित मेमरी शोधण्यासाठी सेमॅंटिक सर्चचा वापर

## वास्तविक-जगातील उपयोग:
- **ग्राहक सेवा**: ग्राहकांचा इतिहास आणि पसंती लक्षात ठेवणे
- **वैयक्तिक सहाय्यक**: दिवस किंवा आठवडे यामध्ये संदर्भ टिकवून ठेवणे
- **आरोग्यसेवा**: रुग्णांची माहिती आणि पसंती ट्रॅक करणे
- **शिक्षण**: विद्यार्थ्यांची प्रगती आणि शिकण्याच्या शैली लक्षात ठेवणे
- **ई-कॉमर्स**: इतिहासावर आधारित वैयक्तिक खरेदी अनुभव प्रदान करणे

## पुढील पावले:
- वेळ-संवेदनशील माहिती साठी मेमरी कालबाह्यता लागू करणे
- मेमरी महत्त्व स्कोअरिंग जोडणे
- सामायिक मेमरीसह बहु-एजंट प्रणाली तयार करणे
- एंटरप्राइझ वापरासाठी CRM प्रणालींसह एकत्रीकरण करणे
- मेमरी आवृत्तीकरण आणि ऑडिट ट्रेल्स जोडणे



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
